In [26]:
#accessing netcdf files from Hector's folders
import os, glob, sys
sys.path.append("//nobackup//amondal//Python//Hector_Python_Scripts")
sys.path.append("//nobackup/amondal//Python//mit_geos_analysis")
#from mds_store import openmdsdataset
#playing with netcdf - xarray
import numpy as np
import xarray as xr
import dask.array as da
import matplotlib.pyplot as plt
# How about visualization?
%matplotlib inline
import netCDF4
import xgcm
from netCDF4 import Dataset
#from xmitgcm import open_mdsdataset
from GetNCDataSet import getMITNCDataSet
from MIT_xr_cwt_dateloc_fol import loadMITData
from MITllc2160Depth import *



from dask.distributed import Client
client = Client(memory_limit='90GB',n_workers = 40, threads_per_worker=1)

In [46]:
sys.path.append("//nobackup/amondal//Python//xmitgcm//xmitgcm")
from xmitgcm.mds_store import open_mdsdataset
sys.path.append("//nobackup/amondal//Python//xrft//xrft")
from xrft.xrft import *
from xrft.detrend import *

In [28]:
%time depthToZlayer(200)

CPU times: user 6.77 ms, sys: 5.23 ms, total: 12 ms
Wall time: 21.9 ms


28

In [29]:
def _e1e2(navlon,navlat):
    earthrad = 6371229     # mean earth radius (m)

    deg2rad = np.pi / 180.

    lam = navlon

    phi = navlat

    djlam,dilam = np.gradient(lam)

    djphi,diphi = np.gradient(phi)

    e1 = earthrad * deg2rad * np.sqrt( (dilam * np.cos(deg2rad*phi))**2. + diphi**2.)

    e2 = earthrad * deg2rad * np.sqrt( (djlam * np.cos(deg2rad*phi))**2. + djphi**2.)

    return e1,e2

 

#########################
"""
Perform a spatial interpolation if required; return da_reg (km/m and days/hrs coordinates)
da - array with lat, lon, time (degrees and hours)
"""
def regularizeCoordinates(da,interp=None, timeunits='days', spaceunits ='km'):
    davals = da.values
    time = da.time.values
    lon = da.lon.values
    lat = da.lat.values
    lon_mesh,lat_mesh = np.meshgrid(lat, lon)
    e1,e2 = _e1e2(lon_mesh,lat_mesh)

    if ( spaceunits== 'km'):
        length_factor = 1000
    else:
        length_factor = 1
        
    if ( timeunits == 'days' ):
        time_factor = 24
    else:
        time_factor = 1
    x1d_in = (e1[0,:].cumsum() - e1[0,0] ) / length_factor # convert from m to km

    y1d_in = (e2[:,0].cumsum() - e2[0,0] ) / length_factor # convert from m to km
    
    da_met = xr.DataArray(data = davals, dims = ['time','xdim','ydim'], coords=[time / time_factor, x1d_in, y1d_in])
    
    if interp is not None:
        x1d_new = np.linspace(x1d_in.min(), x1d_in.max(), len(x1d_in))
        y1d_new = np.linspace(y1d_in.min(), y1d_in.max(), len(y1d_in))
        da_reg = da_met.interp(xdim = x1d_new, ydim = y1d_new, method=interp)
    #x2d_in,y2d_in = np.meshgrid(x1d_in,y1d_in)
    else:
        da_reg = da_met
   
    return da_reg #we can work on getting this to interpolate next

In [30]:
T15 = getMITNCDataSet('/nobackup/amondal/NCData/20211103_multicoretest_3month_noland_attempt2/', 'Theta', 15,15 )

Time combined dataset has been opened.
I concatenated the first Z-layer


In [31]:
T15c = T15.compute()
T15c

<xarray.Dataset>
Dimensions:  (lat: 300, lon: 375, time: 2208)
Coordinates:
  * lat      (lat) float64 30.0 30.04 30.08 30.12 ... 41.84 41.88 41.92 41.96
  * lon      (lon) float64 -60.0 -59.96 -59.92 -59.88 ... -45.12 -45.08 -45.04
Dimensions without coordinates: time
Data variables:
    Theta    (time, lat, lon) float64 20.03 20.03 20.06 ... 15.25 15.21 15.26

In [32]:
W = getMITNCDataSet('/nobackup/amondal/NCData/20211116_QTprime_openocean_3month/', 'W', 15,15)

Time combined dataset has been opened.
I concatenated the first Z-layer


In [33]:
Wc = W.compute()
Wc

<xarray.Dataset>
Dimensions:  (lat: 300, lon: 375, time: 2208)
Coordinates:
  * lat      (lat) float64 30.0 30.04 30.08 30.12 ... 41.84 41.88 41.92 41.96
  * lon      (lon) float64 -60.0 -59.96 -59.92 -59.88 ... -45.12 -45.08 -45.04
Dimensions without coordinates: time
Data variables:
    W        (time, lat, lon) float64 -0.0001791 -0.0001791 ... 5.382e-05

In [34]:
Theta15 = T15c.Theta
W15 = Wc.W

In [35]:
Theta15

<xarray.DataArray 'Theta' (time: 2208, lat: 300, lon: 375)>
array([[[20.03393936, 20.03393936, 20.05987549, ..., 20.07913017,
         20.11499786, 20.10113144],
        [20.0337429 , 20.0337429 , 20.04794693, ..., 20.07556152,
         20.11726761, 20.10302162],
        [20.03877068, 20.03877068, 20.04024124, ..., 20.08831024,
         20.11519623, 20.10379028],
        ...,
        [ 9.50843906,  9.50843906,  9.49765587, ..., 14.62988758,
         14.61009407, 14.59073544],
        [ 9.64250469,  9.64250469,  9.61566925, ..., 14.45315361,
         14.45778275, 14.46081638],
        [10.18348598, 10.18348598, 10.12649059, ..., 14.34997654,
         14.35819054, 14.36782169]],

       [[20.03316498, 20.03316498, 20.06603813, ..., 20.08368874,
         20.11501312, 20.10103607],
        [20.03221321, 20.03221321, 20.05183029, ..., 20.07873154,
         20.11715317, 20.10276794],
        [20.03635406, 20.03635406, 20.04163742, ..., 20.09045601,
         20.11499977, 20.10357475],
...
        [13.47716427, 13.47716427, 13.62457371, ..., 15.22689819,
         15.16504765, 15.27029514],
        [13.56757164, 13.56757164, 13.71325207, ..., 15.26884174,
         15.20705032, 15.27132607],
        [13.69296646, 13.69296646, 13.81317997, ..., 15.26718616,
         15.21602631, 15.26326561]],

       [[20.59471512, 20.59471512, 20.63847542, ..., 20.28848839,
         20.44502068, 20.21613121],
        [20.64384651, 20.64384651, 20.71489716, ..., 20.3376255 ,
         20.42549324, 20.25374603],
        [20.71295738, 20.71295738, 20.71903992, ..., 20.19218826,
         20.24973869, 20.28374481],
        ...,
        [13.43190861, 13.43190861, 13.59977341, ..., 15.23666954,
         15.16348839, 15.25693226],
        [13.51799202, 13.51799202, 13.69208145, ..., 15.2660923 ,
         15.21285534, 15.27589893],
        [13.63515568, 13.63515568, 13.80089664, ..., 15.25488758,
         15.2074852 , 15.26331234]]])
Coordinates:
  * lat      (lat) float64 30.0 30.04 30.08 30.12 ... 41.84 41.88 41.92 41.96
  * lon      (lon) float64 -60.0 -59.96 -59.92 -59.88 ... -45.12 -45.08 -45.04
Dimensions without coordinates: time

In [36]:
Theta15_i = Theta15.interp(lon = W15.lon, method = 'linear')

In [37]:
Theta15_i

<xarray.DataArray 'Theta' (time: 2208, lat: 300, lon: 375)>
array([[[20.03393936, 20.03393936, 20.05987549, ..., 20.07913017,
         20.11499786, 20.10113144],
        [20.0337429 , 20.0337429 , 20.04794693, ..., 20.07556152,
         20.11726761, 20.10302162],
        [20.03877068, 20.03877068, 20.04024124, ..., 20.08831024,
         20.11519623, 20.10379028],
        ...,
        [ 9.50843906,  9.50843906,  9.49765587, ..., 14.62988758,
         14.61009407, 14.59073544],
        [ 9.64250469,  9.64250469,  9.61566925, ..., 14.45315361,
         14.45778275, 14.46081638],
        [10.18348598, 10.18348598, 10.12649059, ..., 14.34997654,
         14.35819054, 14.36782169]],

       [[20.03316498, 20.03316498, 20.06603813, ..., 20.08368874,
         20.11501312, 20.10103607],
        [20.03221321, 20.03221321, 20.05183029, ..., 20.07873154,
         20.11715317, 20.10276794],
        [20.03635406, 20.03635406, 20.04163742, ..., 20.09045601,
         20.11499977, 20.10357475],
...
        [13.47716427, 13.47716427, 13.62457371, ..., 15.22689819,
         15.16504765, 15.27029514],
        [13.56757164, 13.56757164, 13.71325207, ..., 15.26884174,
         15.20705032, 15.27132607],
        [13.69296646, 13.69296646, 13.81317997, ..., 15.26718616,
         15.21602631, 15.26326561]],

       [[20.59471512, 20.59471512, 20.63847542, ..., 20.28848839,
         20.44502068, 20.21613121],
        [20.64384651, 20.64384651, 20.71489716, ..., 20.3376255 ,
         20.42549324, 20.25374603],
        [20.71295738, 20.71295738, 20.71903992, ..., 20.19218826,
         20.24973869, 20.28374481],
        ...,
        [13.43190861, 13.43190861, 13.59977341, ..., 15.23666954,
         15.16348839, 15.25693226],
        [13.51799202, 13.51799202, 13.69208145, ..., 15.2660923 ,
         15.21285534, 15.27589893],
        [13.63515568, 13.63515568, 13.80089664, ..., 15.25488758,
         15.2074852 , 15.26331234]]])
Coordinates:
  * lat      (lat) float64 30.0 30.04 30.08 30.12 ... 41.84 41.88 41.92 41.96
  * lon      (lon) float64 -60.0 -59.96 -59.92 -59.88 ... -45.12 -45.08 -45.04
Dimensions without coordinates: time

In [38]:
Theta15_reg = regularizeCoordinates(Theta15_i,'linear',timeunits = 'hours')
W15_reg = regularizeCoordinates(W15, 'linear', timeunits = 'hours')

In [39]:
Theta15_reg.to_netcdf('/nobackup/amondal/NCData/Theta15_reg_for_Hector.nc4')
W15_reg.to_netcdf('/nobackup/amondal/NCData/W15_reg_for_Hector.nc4')

In [40]:
Theta15_reg.ydim.diff('ydim').mean()

<xarray.DataArray 'ydim' ()>
array(4.44795694)

In [41]:
Theta15_reg_w = Theta15_reg[0:1250]
Theta15_reg_s = Theta15_reg[1251:]
W15_reg_w = W15_reg[0:1250]
W15_reg_s = W15_reg[1251:]

In [42]:
Theta15_reg_w.xdim.diff('xdim').mean()

<xarray.DataArray 'xdim' ()>
array(2.22397847)

In [47]:
Theta15_reg2_w = xrft.detrend(Theta15_reg_w, dim = ['time'], detrend_type = 'constant')
W15_reg2_w = xrft.detrend(W15_reg_w, dim = 'time', detrend_type = 'constant')
Theta15_reg3_w = xrft.detrend(Theta15_reg2_w, dim = ['xdim','ydim'], detrend_type = 'constant')
W15_reg3_w = xrft.detrend(W15_reg2_w, dim = ['xdim','ydim'], detrend_type = 'constant')

Theta15_reg2_s = xrft.detrend(Theta15_reg_s, dim = ['time'], detrend_type = 'constant')
W15_reg2_s = xrft.detrend(W15_reg_s, dim = 'time', detrend_type = 'constant')
Theta15_reg3_s = xrft.detrend(Theta15_reg2_s, dim = ['xdim','ydim'], detrend_type = 'constant')
W15_reg3_s = xrft.detrend(W15_reg2_s, dim = ['xdim','ydim'], detrend_type = 'constant')

Lx_w = Theta15_reg3_w.xdim.size
Ly_w = Theta15_reg3_w.ydim.size
Lt_w = Theta15_reg3_w.time.size

Lx_s = Theta15_reg3_s.xdim.size
Ly_s = Theta15_reg3_s.ydim.size
Lt_s = Theta15_reg3_s.time.size

dx_w = Theta15_reg3_w.xdim.diff('xdim').mean()
dy_w = Theta15_reg3_w.ydim.diff('ydim').mean()
dt_w = Theta15_reg3_w.time.diff('time').mean()

dx_s = Theta15_reg3_s.xdim.diff('xdim').mean()
dy_s = Theta15_reg3_s.ydim.diff('ydim').mean()
dt_s = Theta15_reg3_s.time.diff('time').mean()
"""
Lx_w = Theta15_reg2_w.xdim.max() - Theta15_reg2_w.xdim.min()
Ly_w = Theta15_reg2_w.ydim.max() - Theta15_reg2_w.ydim.min()
Lt_w = Theta15_reg2_w.time.max() - Theta15_reg2_w.time.min()

Lx_s = Theta15_reg2_s.xdim.max() - Theta15_reg2_s.xdim.min()
Ly_s = Theta15_reg2_s.ydim.max() - Theta15_reg2_s.ydim.min()
Lt_s = Theta15_reg2_s.time.max() - Theta15_reg2_s.time.min()
"""

NameError: name 'xrft' is not defined

In [ ]:
Theta_time_w = (Theta15_reg3_s * W15_reg3_s * 4.2*10**6).mean('time')

In [ ]:
%matplotlib inline
levels = np.arange(-200, 200, 2)
Theta_time_w.plot(levels = levels)


In [ ]:
Theta_time_w_space = (Theta15_reg3_s * W15_reg3_s * 4.2*10**6).mean(['xdim', 'ydim'])

In [ ]:
%matplotlib inline 
Theta_time_w_space.plot()

In [ ]:
Theta_time_w_spacetime = (Theta15_reg3_s * W15_reg3_s * 4.2*10**6).mean()
Theta_time_w_spacetime

In [ ]:
_,T15_win_w = xrft.xrft._apply_window(Theta15_reg3_w,['xdim','ydim','time'], window_type='tukey')
_,W15_win_w = xrft.xrft._apply_window(W15_reg3_w ,['xdim','ydim','time'], window_type='tukey')
_,T15_win_s = xrft.xrft._apply_window(Theta15_reg3_s,['xdim','ydim','time'], window_type='tukey')
_,W15_win_s = xrft.xrft._apply_window(W15_reg3_s ,['xdim','ydim','time'], window_type='tukey')

In [ ]:
#Inner Product check for Plancherel's check
Norm_w =  (T15_win_w * W15_win_w * dx_w * dy_w * dt_w).sum()
Norm_s =  (T15_win_s * W15_win_s * dx_s * dy_s * dt_s).sum()

ISOTROPIC_CROSS_SPECTRUM doesn't work unless we transform xdim and ydim to rdim before the fft


Also, if you ask it to chunk in one dimension, it default chunks in all dimensions - so don't bother trying to do all 3 dimensions at once

In [ ]:
T15_w = xrft.xrft.fft(Theta15_reg3_w, dim = ['ydim'], real_dim = 'ydim', window = 'tukey',  true_amplitude = True, true_phase = True, truncate = True, )
W15_w = xrft.xrft.fft(W15_reg3_w, dim = ['ydim'], real_dim = 'ydim',  window = 'tukey',  true_amplitude = True,  true_phase = True,truncate = True, )
T15_w2 = xrft.xrft.fft(T15_w, dim= 'xdim', real_dim = 'xdim', window = 'tukey',  true_amplitude = True, true_phase = True,truncate = True, )
W15_w2 = xrft.xrft.fft(W15_w, dim= 'xdim', real_dim = 'xdim', window = 'tukey', true_amplitude = True, true_phase = True,truncate = True, )

T15_s = xrft.xrft.fft(Theta15_reg3_s, dim = ['ydim'], real_dim = 'ydim', window = 'tukey', true_amplitude = True, true_phase = True,truncate = True, )
W15_s = xrft.xrft.fft(W15_reg3_s, dim = ['ydim'], real_dim = 'ydim',  window = 'tukey', true_amplitude = True,  true_phase = True,truncate = True, )
T15_s2 = xrft.xrft.fft(T15_s, dim= 'xdim', real_dim = 'xdim', window = 'tukey', true_amplitude = True, true_phase = True,truncate = True, )
W15_s2 = xrft.xrft.fft(W15_s, dim= 'xdim', real_dim = 'xdim', window = 'tukey', true_amplitude = True, true_phase = True,truncate = True, )



In [ ]:
Lx_w

In [ ]:
T15_s2.time.size

In [ ]:
"""
# let's attempt Welch's method:
Tn_w = T15_w.time.size
Wn_w = W15_w.time.size
segsize = 7
easychunknum = (segsize + 1) / 2
overlapnum = easychunknum - 1
length = int(Tn_w / easychunknum)
append_da = T15_w[int(easychunknum / 2): int(easychunknum / 2) + length * overlapnum]
T15_w_append = xr.concat([T15_w, append_da], compat = "identical", dim='time')
#finish welch's method off
"""
"""
# this does Bartlett's method 
Tn_w = T15_w2.time.size
Wn_w = W15_w2.time.size
T15_wkl_w = xrft.xrft.fft(T15_w2.chunk({'time':int(Tn_w/5)}), dim = 'time', real_dim = 'time', window = 'tukey', window_correction = True, true_amplitude = True, truncate = True, chunks_to_segments=True).compute()
W15_wkl_w = xrft.xrft.fft(W15_w2.chunk({'time':int(Wn_w/5)}), dim = 'time', real_dim = 'time',  window = 'tukey', window_correction = True, true_amplitude = True,  truncate = True, chunks_to_segments=True).compute()


Tn_s = T15_s2.time.size
Wn_s = W15_s2.time.size
T15_wkl_s = xrft.xrft.fft(T15_s2[0:955].chunk({'time':int(Tn_s/5)}), dim = 'time', real_dim = 'time', window = 'tukey', window_correction = True, true_amplitude = True, truncate = True, chunks_to_segments=True).compute()
W15_wkl_s = xrft.xrft.fft(W15_s2[0:955].chunk({'time':int(Wn_s/5)}), dim = 'time', real_dim = 'time',  window = 'tukey', window_correction = True, true_amplitude = True,  truncate = True, chunks_to_segments=True).compute()
"""
T15_wkl_w = xrft.xrft.fft(T15_w2, dim = 'time', real_dim = 'time', window = 'tukey', true_amplitude = True, true_phase = True,truncate = True).compute()
W15_wkl_w = xrft.xrft.fft(W15_w2, dim = 'time', real_dim = 'time',  window = 'tukey', true_amplitude = True,  true_phase = True,truncate = True).compute()
dkw = T15_wkl_w.freq_xdim.diff('freq_xdim').mean()
dlw = T15_wkl_w.freq_ydim.diff('freq_ydim').mean()
domw = T15_wkl_w.freq_time.diff('freq_time').mean()

T15_wkl_s = xrft.xrft.fft(T15_s2[0:955], dim = 'time', real_dim = 'time', window = 'tukey', true_amplitude = True,true_phase = True, truncate = True).compute()
W15_wkl_s = xrft.xrft.fft(W15_s2[0:955], dim = 'time', real_dim = 'time',  window = 'tukey', true_amplitude = True,true_phase = True,  truncate = True).compute()
dks = T15_wkl_s.freq_xdim.diff('freq_xdim').mean()
dls = T15_wkl_s.freq_ydim.diff('freq_ydim').mean()
doms = T15_wkl_s.freq_time.diff('freq_time').mean()

In [ ]:
doms

In [ ]:
T15_wkl_w_conj = np.conjugate(T15_wkl_w)
T15_wkl_s_conj = np.conjugate(T15_wkl_s)

In [ ]:
Qp15_w = W15_wkl_w * T15_wkl_w_conj * (dkw * dlw * domw)# / (Lx_w * Ly_w * Lt_w)#/ (Lx_w * Ly_w * Lt_w)**2 / (dkw * dlw * domw)**2  #include normalization
Qp15_s = W15_wkl_s * T15_wkl_s_conj * (dks * dls * doms)  # / (Lx_s * Ly_s * Lt_s)#/ (Lx_s * Ly_s * Lt_s)**2 / (dks * dls * doms)**2

In [ ]:
# norm for Plancherel's theorem check
KNorm_w = np.real(Qp15_w.sum()) #/ (Lx_w * Ly_w * Lt_w)
KNorm_s = np.real(Qp15_s.sum()) #/ (Lx_s * Ly_s * Lt_s)

In [ ]:
Error_w = (KNorm_w - Norm_w) / Norm_w
Error_s = (KNorm_s - Norm_s) / Norm_s

In [ ]:
Error_w

In [ ]:
(Lx_w * Ly_w * Lt_w) 

In [ ]:
1 / (Lx_w * Ly_w * Lt_w)

In [ ]:
%%time 
Qpc15_w = Qp15_w#.mean('time_segment')
Qpc15_s = Qp15_s#.mean('time_segment')

In [ ]:
Qp15_w_iso = xrft.xrft.isotropize(np.real(Qpc15_w), ['freq_xdim', 'freq_ydim'], truncate = True)
Qp15_s_iso = xrft.xrft.isotropize(np.real(Qpc15_s), ['freq_xdim', 'freq_ydim'], truncate = True)

In [ ]:
Eisop15_w = Qp15_w_iso.values[1:]
omp15_w = Qp15_w_iso.freq_time[1:].values
kisop15_w = Qp15_w_iso.freq_r.values

Eisop15_s = Qp15_s_iso.values[1:]
omp15_s = Qp15_s_iso.freq_time[1:].values
kisop15_s = Qp15_s_iso.freq_r.values

In [ ]:
rhoConst = 1027.5 #kg/m^3
Cp = 4.185*(10**3) # J / (kg K)

In [ ]:
%matplotlib inline
from matplotlib.colors import LogNorm
fig = plt.figure(figsize = (15,5))
ax1 = fig.add_subplot(122)
cs = plt.pcolormesh(kisop15_w, omp15_w, (Cp*rhoConst*1*Eisop15_w*kisop15_w[None,...]*omp15_w[...,None]), shading='gouraud', cmap='seismic')#, norm=LogNorm())
ax1.set_yscale('log')
ax1.set_xscale('log')
plt.title(r'Vertical Heat Flux Cospectrum (45m) (March 1 - April 20)', size = 16)
plt.xlabel(r'Wavenumber[cpkm]', size=16)
plt.ylabel(r'Frequency [cph]', size = 16)
#plt.clim([-.0,.001])
plt.colorbar()
plt.show()

$ K \times \omega \times Re\big[\rho C_p \hat{W}' \hat{T}'\big]$ - 3month period split into 2 periods,  5 windows (Bartlett's method, not Welch's)

Does the spatial scale make any sense?

In [ ]:
%matplotlib inline
from matplotlib.colors import LogNorm
fig = plt.figure(figsize = (15,5))
ax1 = fig.add_subplot(122)
cs = plt.pcolormesh(kisop15_s, omp15_s, (Cp*rhoConst*1*Eisop15_s*kisop15_s[None,...]*omp15_s[...,None]), shading='gouraud', cmap='seismic')#, norm=LogNorm())
ax1.set_yscale('log')
ax1.set_xscale('log')
plt.title(r'Vertical Heat Flux Cospectrum (45m) (April 20 - May 31)', size = 16)
plt.xlabel(r'Wavenumber[cpkm]', size=16)
plt.ylabel(r'Frequency [cph]', size = 16)
plt.clim([-.001, .001])
plt.colorbar()
plt.show()

In [ ]:
T45 = getMITNCDataSet('/nobackup/amondal/NCData/20211116_QTprime_openocean_3month/', 'Theta', 45,45 )

In [ ]:
T45c = T45.compute()
T45c

In [ ]:
W = getMITNCDataSet('/nobackup/amondal/NCData/20211116_QTprime_openocean_3month/', 'W', 45,45)

In [ ]:
Wc = W.compute()
Wc

In [ ]:
Theta45 = T45c.Theta
W45 = Wc.W

In [ ]:
Theta15

In [ ]:
Theta45_i = Theta45.interp(lon = W45.lon, method = 'linear')

In [ ]:
Theta45_i

In [ ]:
Theta45_reg = regularizeCoordinates(Theta45_i,'linear',timeunits = 'hours')
W45_reg = regularizeCoordinates(W45, 'linear', timeunits = 'hours')

In [ ]:
Theta45_reg_w = Theta45_reg[0:1250]
Theta45_reg_s = Theta45_reg[1251:]
W45_reg_w = W45_reg[0:1250]
W45_reg_s = W45_reg[1251:]

In [ ]:
Theta45_reg_w

In [ ]:
Theta45_reg2_w = xrft.detrend(Theta45_reg_w, dim = ['time'], detrend_type = 'constant')
W45_reg2_w = xrft.detrend(W45_reg_w, dim = 'time', detrend_type = 'constant')
Theta45_reg3_w = xrft.detrend(Theta45_reg2_w, dim = ['xdim','ydim'], detrend_type = 'constant')
W45_reg3_w = xrft.detrend(W45_reg2_w, dim = ['xdim','ydim'], detrend_type = 'constant')

Theta45_reg2_s = xrft.detrend(Theta45_reg_s, dim = ['time'], detrend_type = 'constant')
W45_reg2_s = xrft.detrend(W45_reg_s, dim = 'time', detrend_type = 'constant')
Theta45_reg3_s = xrft.detrend(Theta45_reg2_s, dim = ['xdim','ydim'], detrend_type = 'constant')
W45_reg3_s = xrft.detrend(W45_reg2_s, dim = ['xdim','ydim'], detrend_type = 'constant')

ISOTROPIC_CROSS_SPECTRUM doesn't work unless we transform xdim and ydim to rdim before the fft


Also, if you ask it to chunk in one dimension, it default chunks in all dimensions - so don't bother trying to do all 3 dimensions at once

In [ ]:
T45_w = xrft.xrft.fft(Theta45_reg3_w, dim = ['ydim'], real_dim = 'ydim', window = 'tukey', true_amplitude = True, true_phase = True, truncate = True, )
W45_w = xrft.xrft.fft(W45_reg3_w, dim = ['ydim'], real_dim = 'ydim',  window = 'tukey',  true_amplitude = True,  true_phase = True, truncate = True, )
T45_w2 = xrft.xrft.fft(T45_w, dim= 'xdim', real_dim = 'xdim', window = 'tukey', true_amplitude = True, true_phase = True, truncate = True, )
W45_w2 = xrft.xrft.fft(W45_w, dim= 'xdim', real_dim = 'xdim', window = 'tukey',true_amplitude = True, true_phase = True, truncate = True, )

T45_s = xrft.xrft.fft(Theta45_reg3_s, dim = ['ydim'], real_dim = 'ydim', window = 'tukey',true_amplitude = True, true_phase = True, truncate = True, )
W45_s = xrft.xrft.fft(W45_reg3_s, dim = ['ydim'], real_dim = 'ydim',  window = 'tukey',  true_amplitude = True,  true_phase = True, truncate = True, )
T45_s2 = xrft.xrft.fft(T45_s, dim= 'xdim', real_dim = 'xdim', window = 'tukey',  true_amplitude = True, true_phase = True, truncate = True, )
W45_s2 = xrft.xrft.fft(W45_s, dim= 'xdim', real_dim = 'xdim', window = 'tukey',  true_amplitude = True, true_phase = True, truncate = True, )



In [ ]:
T15_w

In [ ]:
T15_s2.time.size

In [ ]:
"""
# let's attempt Welch's method:
Tn_w = T15_w.time.size
Wn_w = W15_w.time.size
segsize = 7
easychunknum = (segsize + 1) / 2
overlapnum = easychunknum - 1
length = int(Tn_w / easychunknum)
append_da = T15_w[int(easychunknum / 2): int(easychunknum / 2) + length * overlapnum]
T15_w_append = xr.concat([T15_w, append_da], compat = "identical", dim='time')
#finish welch's method off
"""
"""
# this does Bartlett's method 
Tn_w = T15_w2.time.size
Wn_w = W15_w2.time.size
T15_wkl_w = xrft.xrft.fft(T15_w2.chunk({'time':int(Tn_w/5)}), dim = 'time', real_dim = 'time', window = 'tukey', true_amplitude = True, truncate = True, chunks_to_segments=True).compute()
W15_wkl_w = xrft.xrft.fft(W15_w2.chunk({'time':int(Wn_w/5)}), dim = 'time', real_dim = 'time',  window = 'tukey',true_amplitude = True,  truncate = True, chunks_to_segments=True).compute()


Tn_s = T15_s2.time.size
Wn_s = W15_s2.time.size
T15_wkl_s = xrft.xrft.fft(T15_s2[0:955].chunk({'time':int(Tn_s/5)}), dim = 'time', real_dim = 'time', window = 'tukey',  true_amplitude = True, truncate = True, chunks_to_segments=True).compute()
W15_wkl_s = xrft.xrft.fft(W15_s2[0:955].chunk({'time':int(Wn_s/5)}), dim = 'time', real_dim = 'time',  window = 'tukey',  true_amplitude = True,  truncate = True, chunks_to_segments=True).compute()
"""
T45_wkl_w = xrft.xrft.fft(T45_w2, dim = 'time', real_dim = 'time', window = 'tukey',true_amplitude = True, true_phase = True, truncate = True).compute()
W45_wkl_w = xrft.xrft.fft(W45_w2, dim = 'time', real_dim = 'time',  window = 'tukey', true_amplitude = True,  true_phase = True, truncate = True).compute()
dkw = T45_wkl_w.freq_xdim.diff('freq_xdim').mean()
dlw = T45_wkl_w.freq_ydim.diff('freq_ydim').mean()
domw = T45_wkl_w.freq_time.diff('freq_time').mean()

T45_wkl_s = xrft.xrft.fft(T45_s2[0:955], dim = 'time', real_dim = 'time', window = 'tukey', true_amplitude = True, true_phase = True, truncate = True).compute()
W45_wkl_s = xrft.xrft.fft(W45_s2[0:955], dim = 'time', real_dim = 'time',  window = 'tukey', true_amplitude = True, true_phase = True, truncate = True).compute()
dks = T45_wkl_s.freq_xdim.diff('freq_xdim').mean()
dls = T45_wkl_s.freq_ydim.diff('freq_ydim').mean()
doms = T45_wkl_s.freq_time.diff('freq_time').mean()

In [ ]:
T45_wkl_w_conj = np.conjugate(T45_wkl_w)
T45_wkl_s_conj = np.conjugate(T45_wkl_s)

In [ ]:
Qp45_w = W45_wkl_w * T45_wkl_w_conj * ( dkw * dlw * domw )
Qp45_s = W45_wkl_s * T45_wkl_s_conj * ( dks * dls * doms )

In [ ]:
%%time 
Qpc45_w = Qp45_w#.mean('time_segment')
Qpc45_s = Qp45_s#.mean('time_segment')

In [ ]:
Qp45_w_iso = xrft.xrft.isotropize(np.real(Qpc45_w), ['freq_xdim', 'freq_ydim'], truncate = True)
Qp45_s_iso = xrft.xrft.isotropize(np.real(Qpc45_s), ['freq_xdim', 'freq_ydim'], truncate = True)

In [ ]:
Eisop45_w = Qp45_w_iso.values[1:]
omp45_w = Qp45_w_iso.freq_time[1:].values
kisop45_w = Qp45_w_iso.freq_r.values

Eisop45_s = Qp45_s_iso.values[1:]
omp45_s = Qp45_s_iso.freq_time[1:].values
kisop45_s = Qp45_s_iso.freq_r.values

In [ ]:
rhoConst = 1027.5 #kg/m^3
Cp = 1*(10**3) # J / (kg K)

In [ ]:
%matplotlib inline
from matplotlib.colors import LogNorm
fig = plt.figure(figsize = (15,5))
ax1 = fig.add_subplot(122)
cs = plt.pcolormesh(kisop45_w, omp45_w, (Cp*rhoConst*1*Eisop45_w*kisop45_w[None,...]*omp45_w[...,None]), shading='gouraud', cmap='seismic')#, norm=LogNorm())
ax1.set_yscale('log')
ax1.set_xscale('log')
plt.title(r'Vertical Heat Flux Cospectrum (750m) (March 1 - April 20)', size = 16)
plt.xlabel(r'Wavenumber[cpkm]', size=16)
plt.ylabel(r'Frequency [cph]', size = 16)
plt.clim([-0.002, 0.002])
plt.colorbar()
plt.show()

$ K \times \omega \times Re\big[\rho C_p \hat{W}' \hat{T}'\big]$ - 3month period split into 2 periods,  5 windows (Bartlett's method, not Welch's)

Does the spatial scale make any sense?

In [ ]:
%matplotlib inline
from matplotlib.colors import LogNorm
fig = plt.figure(figsize = (15,5))
ax1 = fig.add_subplot(122)
cs = plt.pcolormesh(kisop45_s, omp45_s, (Cp*rhoConst*1*Eisop45_s*kisop45_s[None,...]*omp45_s[...,None]), shading='gouraud', cmap='seismic')#, norm=LogNorm())
ax1.set_yscale('log')
ax1.set_xscale('log')
plt.title(r'Vertical Heat Flux Cospectrum (750m) (April 20 - May 31)', size = 16)
plt.xlabel(r'Wavenumber[cpkm]', size=16)
plt.ylabel(r'Frequency [cph]', size = 16)
plt.clim([-0.002,0.002])
plt.colorbar()
plt.show()

Let me create a section for checking out the KE spectrum. That will go below here and I will calculate those spectra

 Patrice recommends trying to look at SSH first. Let's also try Parseval's theorem with super simple spectra.

In [ ]:
Nx = 200
dx = np.random.rand()
#da = xr.DataArray(np.random.rand(Nx) + 1j * np.random.rand(Nx), dims="x", coords={"x": dx * (np.arange(-Nx // 2, -Nx // 2 + Nx) + np.random.randint(-Nx // 2, Nx // 2))},)
#db = xr.DataArray(np.random.rand(Nx) + 1j*np.random.rand(Nx), dims="x", coords={"x":dx * (np.arange(-Nx//2, -Nx//2 + Nx) + np.random.randint(-Nx // 2, Nx // 2))})
da = xr.DataArray(np.random.rand(Nx) , dims="x", coords={"x": dx * (np.arange(-Nx // 2, -Nx // 2 + Nx) + np.random.randint(-Nx // 2, Nx // 2))},)
db = xr.DataArray(np.random.rand(Nx) , dims="x", coords={"x":dx * (np.arange(-Nx//2, -Nx//2 + Nx) + np.random.randint(-Nx // 2, Nx // 2))})
da2 = xrft.detrend(da, "x", detrend_type='linear')
db2 = xrft.detrend(db, "x", detrend_type='linear')
win, da3 = xrft.xrft._apply_window(da2, "x", window_type='tukey')
_, db3 = xrft.xrft._apply_window(db2, "x", window_type='tukey')


In [ ]:
%matplotlib inline
da.plot()

In [ ]:
FT = xrft.dft(db3, dim="x", real_dim="x", true_amplitude=True, truncate=True)
#npt.assert_almost_equal(
        #(np.abs(da) ** 2).sum() * dx, (np.abs(FT) ** 2).sum() * FT["freq_x"].spacing
           #            )
#FT2 = xrft.dft(db, dim="x", true_phase=True, true_amplitude=True, truncate=True, window='tukey', detrend='linear')
#CS = xrft.cross_spectrum(db, db, dim='x', real_dim = 'x', true_amplitude=True, truncate=True, window='tukey', window_correction=True, detrend='linear')
ps = xrft.power_spectrum(db3, dim="x", real_dim="x", true_amplitude=True, truncate=True)
FPS = np.abs(FT)**2
f = np.full(FPS["freq_x"].size, 2.0)
if FPS["freq_x"].size % 2 == 0:
    f[0], f[-1] = 1.0
else:
    f[0] = 1.0
FPS = f*FPS

In [ ]:
Normx = (np.abs(db3)**2).sum() * dx
Fnormx = (FPS).sum() * (FT["freq_x"].spacing)
#CSN = np.abs(CS.sum())
Pnormx = ps.sum() #SO DEFINITELY USE POWER SPECTRUM IN THE FUTURE. BUT WITH PLOTTING POWER SPECTRUM, DIVIDE OUT A FACTOR OF DX

In [ ]:
np.abs((Fnormx - Normx) / Normx)

In [ ]:
np.abs((Pnormx - Normx) / Normx)

In [ ]:
db3.var()

In [ ]:
#here I will write code to do my own cross spectrum calculation 
#first, I want to detrend the output
da_d = xrft.detrend(da, "x", detrend_type='linear')
db_d = xrft.detrend(db, "x", detrend_type='linear')
#second, I want to window the output
awin, da_dw = xrft.xrft._apply_window(da_d, "x", window_type='hann')
bwin, db_dw = xrft.xrft._apply_window(db_d, "x", window_type='hann')
#third, I take the real Fourier transform
da_hat = xrft.xrft.fft(da_dw, dim = "x",  real_dim="x", true_phase = True, true_amplitude = True, truncate = True ) 
db_hat = xrft.xrft.fft(db_dw, dim = "x",  real_dim="x", true_phase = True, true_amplitude = True, truncate = True ) 
#fourth, I need to combine the two spectra and appropriately normalize (window correction, 1/L I believe as well)
db_hat_star = np.conj(db_hat)
db_dw_star = np.conj(db_dw)
cospec = da_hat * db_hat_star * 1/(awin*bwin).mean() 
f = np.full(cospec.freq_x.size, 2.0)
if cospec.freq_x.size % 2 == 0:
    f[0], f[-1] = 1.0, 1.0
else:
    f[0] = 1.0
cospec = f*cospec

In [ ]:
k_norm = (np.real(cospec)*da_hat["freq_x"].spacing**2).sum() - np.real(cospec[0])*da_hat["freq_x"].spacing**2 - np.real(cospec[-1])*da_hat["freq_x"].spacing**2
r_norm = (da_d*db_d*dx).sum()* 1/(awin*bwin).mean()

In [ ]:
crosspec = cross_spectrum(da_d, db_d, dim="x", real_dim="x", true_amplitude=True, truncate=True, window ='hann', window_correction=True)

In [ ]:
np.abs((k_norm - r_norm) / (r_norm))

In [ ]:
np.abs((crosspec.sum() - r_norm) / (r_norm))

In [ ]:
k_norm

In [ ]:
r_norm

UPDATES FROM HECTOR MEETING:

COPY XRFT'S CODE AND SEE IF YOU CAN FIX IT


Things to check with Hector:

1. With xrft, spectra matches Parseval theorem unless I do the real_fft. In which case, it doesn't work so well. But this could just be due to whatever transformation the real dim does.
2. WIndow functions and detrending make it harder to get a good match. Do this separately.
3. I don't need a lot of normalization to get Parseval's to work in 1D
4. Should the size of the data matter?
5. Is the approximation for inner product of different functions worse than same function?

In [ ]:
da_hat["freq_x"].spacing

Make it a goal to figure out the depth of integration by the time you get to Caltech!!